### **Data Loading**

In [17]:
!pip install kaggle
!pip install --upgrade kaggle
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c uw-cs480-fall20

Requirement already up-to-date: kaggle in /usr/local/lib/python3.6/dist-packages (1.5.10)


Saving kaggle.json to kaggle (1).json
uw-cs480-fall20.zip: Skipping, found more recently modified local copy (use --force to force download)


In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
!ls
import zipfile
zip_ref = zipfile.ZipFile('uw-cs480-fall20.zip', 'r')
zip_ref.extractall('files')
zip_ref.close()
import pandas as pd
samples=pd.read_csv('/content/files/sample_submission.csv',dtype=str)
test_data=pd.read_csv('/content/files/test.csv')
train_data=pd.read_csv('/content/files/train.csv')
image_dir = '/content/files/suffled-images'
train_data=train_data.reset_index(drop=True)
test_data=test_data.reset_index(drop=True)


 drive	'kaggle (1).json'   sample_data
 files	 kaggle.json	    uw-cs480-fall20.zip


In [20]:
test_data=pd.read_csv('/content/files/test.csv',dtype=str)
train_data=pd.read_csv('/content/files/train.csv',dtype=str)
image_dir = '/content/files/suffled-images'
train=train_data.reset_index(drop=True)
test=test_data.reset_index(drop=True)
train_folder = '/content/files/'
test_folder = '/content/files/'
samples=pd.read_csv('/content/files/sample_submission.csv',dtype=str)




In [21]:
# train.head()

Data Preparation

In [22]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [23]:
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences

Text_train = train['noisyTextDescription'].values
labels = train['category'].values
text_test = test['noisyTextDescription'].values


encoder = LabelBinarizer()
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(Text_train)
tokenizer.fit_on_texts(text_test)

Text_train = tokenizer.texts_to_sequences(Text_train)
text_test = tokenizer.texts_to_sequences(text_test)

vocab_size = len(tokenizer.word_index) + 1 

maxlen = 30
Text_train = pad_sequences(Text_train, padding='post', maxlen=maxlen)
text_test = pad_sequences(text_test, padding='post', maxlen=maxlen)

labels = encoder.fit_transform(labels) 

### **Multilayer Perceptron model**

In [24]:
import keras
from keras.models import Sequential
from keras import layers
embedding_dim = 150
n_classes = 27
model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(n_classes, activation='softmax'))
model.compile(optimizer='rmsprop',
              loss=keras.losses.categorical_crossentropy,
              metrics=['accuracy'])
model.summary()

history = model.fit(Text_train, labels,
                    epochs=5,
                    verbose=1,
                    validation_split=0.15,
                    batch_size=30)
loss, accuracy = model.evaluate(Text_train, labels, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 150)           1314000   
_________________________________________________________________
flatten_1 (Flatten)          (None, 4500)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               2304512   
_________________________________________________________________
dense_6 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_7 (Dense)              (None, 128)               65664     
_________________________________________________________________
dense_8 (Dense)              (None, 32)                4128      
_________________________________________________________________
dense_9 (Dense)              (None, 27)               

In [25]:
model.save('MLP_89.h5')

In [28]:
import numpy as np
model = keras.models.load_model('MLP_89.h5')
arg = np.argmax(model.predict(text_test), axis=-1)
matrix = np.zeros((arg.shape[0],27))
matrix[np.arange(arg.size),arg] = 1
prediction = encoder.inverse_transform(matrix)
# prediction.shape

In [ ]:
# print(model.predict(Text_train))